## Tools

Reading atomic configuraion and caculating the potential energy and forces using NEP. The results are then plotted and the Root Mean Square Error (RMSE) is computed.

In [ ]:
from wizard.io import read_xyz, plot_e, plot_f
from pynep.calculate import NEP
import numpy as np

frames = read_xyz('train.xyz') 
print(len(frames))
calc = NEP('nep.txt')
ed, en, fd, fn = [], [], [], []
for atoms in frames:
    atoms.calc = calc
    en.append(atoms.get_potential_energy() / len(atoms))
    ed.append(atoms.info['energy'] / len(atoms))
    fn.append(atoms.get_forces())
    fd.append(atoms.info['forces'])
ed = np.array(ed)
en = np.array(en)
fd = np.concatenate(fd)
fn = np.concatenate(fn)
plot_e(ed, en)
plot_f(fd, fn)
e_rmse = np.sqrt(np.mean((ed-en)**2)) 
f_rmse = np.sqrt(np.mean((fd-fn)**2))
print(e_rmse)
print(f_rmse)

In [ ]:
"""
# -----------------------------------------------------------------------------
# The following code is sourced from
# pynep:
# https://github.com/bigd4/PyNEP
#
# Copyright (c) 2022 wang laosi, licensed under MIT License
# -----------------------------------------------------------------------------
"""
from pynep.calculate import NEP
from pynep.select import FarthestPointSample
from wizard.io import read_xyz, dump_xyz
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

atoms = read_xyz('train.xyz')
calc = NEP('nep.txt')
des = np.array([np.mean(calc.get_property('descriptor', i), axis=0) for i in atoms])
sampler = FarthestPointSample(min_distance=0.01)
selected_i = sampler.select(des, [])
for i in selected_i:
    dump_xyz('selected_train.xyz', atoms[i])

reducer = PCA(n_components=2)
reducer.fit(des)
proj = reducer.transform(des)
plt.scatter(proj[:,0], proj[:,1], label='all data')
selected_proj = reducer.transform(np.array([des[i] for i in selected_i]))
plt.scatter(selected_proj[:,0], selected_proj[:,1], label='selected data')
plt.legend()
plt.axis('off')
plt.savefig('select.png')